<a href="https://colab.research.google.com/github/Kamaridddin/Data-science/blob/main/diagnosis_for_diabet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [ ]:
url ="https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/04267bce66c3b9b0a2f301a10888a81ed19c9c70/diabetes.csv"
df = pd.read_csv(url)
df.sample(10)

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
def plot_hist(df,feature):
    plt.hist(df[feature], bins = 50)
    plt.xlabel(feature)
    plt.ylabel("Frequency")
    plt.title("{}distribution with hist".format(feature))
    plt.show()

In [ ]:
numericVar = df.columns[:-1]
for n in numericVar:
    plot_hist(df, n)

In [ ]:
def ZEROs(df):
    print("               # of ZEROs  \t Length \t  Percent "  )
    print("------------------------------------------------------"  )

    for i in range(6) :
        feature=df[df.columns[i]]
        ZeroSum=(feature==0).sum()
        Percent=int(round((ZeroSum/(len(feature)))*100))
        print(df.columns[i], "\t:", ZeroSum , "\t\t:", len(feature), "\t \t:", Percent,"% "  )

In [ ]:
ZEROs(df)

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = 0, strategy = 'median')

X = imputer.fit_transform(X)
X = pd.DataFrame(X, columns=df.columns[: -1])
df = pd.concat([X, y], axis=1)
df2 = df

In [ ]:
ZEROs(df)

In [ ]:
pd.options.display.float_format = "{:,.0f}".format
df

In [ ]:
numericVar = df
for n in numericVar:
  plot_hist(df, n)

In [ ]:
round(df.describe(), 1)

 **Outlier Detection**

In [ ]:
def detect_outliers(df, features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile Q1
        Q1 = np.percentile(df[c], 25)
        # 3st quartile Q3
        Q3 = np.percentile(df[c], 75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indices
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indices
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [ ]:
df.loc[detect_outliers(df, ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])]

In [ ]:
#Drop outliers
df = df.drop(detect_outliers(df, ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']), axis=0 ).reset_index(drop=True)

In [ ]:
list1 = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']

In [ ]:
sns.heatmap(df[list1].corr(), annot=True, cmap='Blues', fmt = ".2f")
plt.show()

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap='Blues', fmt=".2f")
plt.show()

**Outliers and Skew check**

In [ ]:
x=X
for i in x:
    g = sns.distplot(x[i], color = "b", label = "Skewness : %.2f"%(x[i].skew()))
    g = g.legend(loc = "best")
    plt.show()

In [ ]:
for i in X:
    g = sns.FacetGrid(df, col = "Outcome")
    g.map(sns.distplot, i, bins= 25)        
    plt.show()

In [ ]:
def plotHistogram(values,label,feature,title):
    sns.set_style("whitegrid")
    plotOne = sns.FacetGrid(values, hue=label,aspect=2)
    plotOne.map(sns.distplot,feature,kde=False)
    plotOne.set(xlim=(0, values[feature].max()))
    plotOne.add_legend()
    plotOne.set_axis_labels(feature, 'Proportion')
    plotOne.fig.suptitle(title)
    plt.show()
for i in X:
    plotHistogram(df,"Outcome",i,' Diagnosis (Blue = Healthy; Orange = Diabetes)')

In [ ]:
for i in X.columns[:-1]:
  sns.factorplot(x="Age", y=i, hue = "Outcome", data = df)
  plt.show()

In [ ]:
Ls = ["Pregnancies", 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

for i in Ls:
  sns.factorplot(x=i, y="Glucose", hue = "Outcome", data=df)
  plt.show()

Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print("X_train", X_train.shape)
print("X_test", X_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [ ]:
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

In [ ]:
log_model.fit(X_train, y_train)
y_pred = log_model.predict(X_test)
y_train_pred = log_model.predict(X_train)

In [ ]:
print("log_model.coef_:", log_model.coef_,  "\nlog_model.intercept_")

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [ ]:
scr=[precision_score, recall_score, accuracy_score, f1_score]
for i in scr:
  print(i(y_train, y_train_pred))

In [ ]:
for i in scr:
  print(i(y_pred, y_test))

In [ ]:
print("f1_score(y_train, y_train_pred, average=weighted) \t :", f1_score(y_train, y_train_pred, average="weighted"))
print("f1_score(y_test, y_pred, average=weighted)\t\t :", f1_score(y_test, y_pred, average="weighted"))

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,plot_roc_curve
confusion_matrix(y_test, y_pred)

In [ ]:
plot_confusion_matrix(log_model, X_test, y_test)

In [ ]:
plot_confusion_matrix(log_model, X_test, y_test, normalize="all")

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
model = LogisticRegression()

In [ ]:
scores = cross_val_score(model, X_train, y_train, cv = 10)
print("Cross- validation mean of accuracy scores", scores.mean())
print("")
print(scores)

In [ ]:
scores = cross_val_score(model, X_test, y_test, cv= 10)
print("Cross- validation mean of accuracy scores", scores.mean())
print("")
print(scores)

In [ ]:
pd.options.display.float_format = "{:,.4f}".format
model = LogisticRegression()

scores = cross_validate(model, X_train, y_train, scoring = ["accuracy", "precision_weighted", "recall_weighted", 
                                                               "f1_weighted"], cv = 10)
df_scores = pd.DataFrame(scores, index= range(1, 11))
df_scores

In [ ]:
df_scores.mean()[2:]

In [ ]:
model = LogisticRegression()

scores = cross_validate(model, X_test, y_test, scoring = ["accuracy", "precision_weighted", "recall_weighted", 
                                                               "f1_weighted"], cv = 10)
df_scores = pd.DataFrame(scores, index= range(1, 11))
df_scores

In [ ]:
df_scores.mean()[2:]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
print(classification_report(y_train, y_train_pred))

**KNN**

In [ ]:
df.head()

In [ ]:
df2.Outcome.value_counts()

In [ ]:
colors = ["green", "purple"]
sns.countplot(df2.Outcome, palette= colors)
plt.show()

In [ ]:
ND = df[df2.Outcome == 0]
D = df[df2.Outcome == 1]

In [ ]:
plt.scatter(D.Age, D.Glucose, color = "red", label = "Bad")
plt.scatter(ND.Age, ND.Glucose, color= "green", label = "Good")
plt.xlabel("Age")
plt.ylabel("Glucose")
plt.legend()
plt.show()

In [ ]:
labels = ["Healthy", "Diabetics"]
explode = [0, 0]
sizes = df2.Outcome.value_counts().values

In [ ]:
plt.figure(figsize=(7, 7))
plt.pie(sizes, explode = explode, labels = labels, autopct = "%1.1f%%",)
plt.title("Percentage of diagnosis diabetics", color = "orange", fontsize=20)
plt.show()

**Correlation Data**

In [ ]:
corr = df2.corr().Outcome
corr[np.argsort(corr, axis= 0)[:-1]]